In [1]:
from pathlib import Path
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
csv_file = Path.cwd().parent / "data" / "output" / "output_141021-17-30-35.csv"
df = pd.read_csv(csv_file)


In [3]:
len(df)


250

In [4]:
frame_dir = Path.cwd().parent / "data" / "output" / "frames"
len(list(frame_dir.glob("*.jpg")))


250

In [11]:
keypoints = [
    list(
        map(
            float,
            re.sub(r"\s+", " ", keypoint.strip()[1:-1]).strip().split(" "),
        )
    )
    for keypoint in df.iloc[212]["keypoints"][2:-2].split("\n ")
]
keypoints = np.array([keypoints])


In [12]:
img_path = Path.cwd().parent / "data" / "input" / "frames" / "out-0212.jpg"
cap = cv2.VideoCapture(str(img_path))
_, img = cap.read()


In [30]:
L_SHOULDER = 5
R_SHOULDER = 6
L_ELBOW = 7
R_ELBOW = 8
L_WRIST = 9
R_WRIST = 10

L_ARM = [L_SHOULDER, L_ELBOW, L_WRIST]
R_ARM = [R_SHOULDER, R_ELBOW, R_WRIST]

l_arm_points = keypoints[0][L_ARM, :]
r_arm_points = keypoints[0][R_ARM, :]


def project_points_onto_original_image(points, image_size):
    if len(points) == 0:
        return []

    points = points.reshape((-1, 2))

    projected_points = np.array(points, dtype=np.float32)

    width, height = image_size[0], image_size[1]
    projected_points[:, 0] *= width
    projected_points[:, 1] *= height

    return projected_points


# https://stackoverflow.com/questions/56710732
def get_arm_angle(points, image_size):
    projected_points = project_points_onto_original_image(points, image_size)
    ba = projected_points[1] - projected_points[0]
    bc = projected_points[1] - projected_points[2]
    print(ba)
    print(bc)
    print(np.dot(ba, bc))
    print(np.linalg.norm(ba))
    print(np.linalg.norm(bc))
    # print(np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc)))

    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if np.isclose(norm_ba, 0) or np.isclose(norm_bc, 0):
        angle = 0
    else:
        angle = np.degrees(
            np.arccos(np.clip(np.dot(ba, bc) / norm_ba / norm_bc, -1, 1))
        )
    return angle


In [17]:
TOMATO = (77, 103, 255)
POINT_RADIUS = 5


def get_image_size(frame):
    image_size = (frame.shape[1], frame.shape[0])
    return image_size


def project_points_onto_original_image(points, image_size):
    if len(points) == 0:
        return []

    points = points.reshape((-1, 2))

    projected_points = np.array(points, dtype=np.float32)

    width, height = image_size[0], image_size[1]
    projected_points[:, 0] *= width
    projected_points[:, 1] *= height

    return projected_points


def draw_keypoints(
    frame,
    keypoints,
    image_size,
    keypoint_dot_color,
    keypoint_dot_radius,
):
    img_keypoints = project_points_onto_original_image(keypoints, image_size)

    for i, keypoint in enumerate(img_keypoints):
        if i in L_ARM or i in R_ARM:
            draw_one_keypoint_dot(
                frame,
                keypoint,
                keypoint_dot_color,
                keypoint_dot_radius,
            )


def draw_one_keypoint_dot(frame, keypoint, keypoint_dot_color, keypoint_dot_radius):
    """Draw single keypoint"""
    cv2.circle(
        frame,
        (int(keypoint[0]), int(keypoint[1])),
        keypoint_dot_radius,
        keypoint_dot_color,
        -1,
    )


def draw_human_keypoints(image, keypoints):
    image_size = get_image_size(image)
    num_persons = keypoints.shape[0]
    for i in range(num_persons):
        draw_keypoints(image, keypoints[i], image_size, TOMATO, POINT_RADIUS)


In [31]:
img_copy = img.copy()
keypoints_copy = keypoints.copy()
# keypoints_copy[0][L_SHOULDER][0] = 0.1
# keypoints_copy[0][L_SHOULDER][1] = 0.1
# keypoints_copy[0][L_ELBOW][0] = 0.2
# keypoints_copy[0][L_ELBOW][1] = 0.2
# keypoints_copy[0][L_WRIST][0] = 0.25
# keypoints_copy[0][L_WRIST][1] = 0.25
l_arm_points = keypoints_copy[0][L_ARM, :]
r_arm_points = keypoints_copy[0][R_ARM, :]

image_size = get_image_size(img)
l_angle = get_arm_angle(l_arm_points, image_size)
r_angle = get_arm_angle(r_arm_points, image_size)
print(l_angle)
print(r_angle)
# draw_human_keypoints(img_copy, keypoints_copy)

# # fig, ax = plt.subplots(1, 2, figsize=(15, 15))
# # ax[0].imshow(img[:, :, ::-1])
# # ax[1].imshow(img_copy[:, :, ::-1])
# fig, ax = plt.subplots(1, 1, figsize=(15, 15))
# ax.imshow(img_copy[:, :, ::-1])


[153.30426   35.377884]
[47.170593 23.585266]
8065.85
157.33337
52.73831
[-70.7558   -11.792648]
[0. 0.]
0.0
71.73179
0.0
13.570393384716995
0
